<h2>Segmenting and clustering the neighborhoods in the city of toronto</h2>

getting neighorhoods with their borough https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

<h3>reading data from web pasge using beautisoup library</h3>

In [1]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import requests
source = requests.get(' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
table = soup.table  # or table = soup.find('table')
table_rows = table.find_all('tr')
for tr in table_rows:
     td = tr.find_all('td')
     row = [i.text for i in td]
     print(row)

[]
['M1A\n', 'Not assigned\n', 'Not assigned\n']
['M2A\n', 'Not assigned\n', 'Not assigned\n']
['M3A\n', 'North York\n', 'Parkwoods\n']
['M4A\n', 'North York\n', 'Victoria Village\n']
['M5A\n', 'Downtown Toronto\n', 'Regent Park, Harbourfront\n']
['M6A\n', 'North York\n', 'Lawrence Manor, Lawrence Heights\n']
['M7A\n', 'Downtown Toronto\n', "Queen's Park, Ontario Provincial Government\n"]
['M8A\n', 'Not assigned\n', 'Not assigned\n']
['M9A\n', 'Etobicoke\n', 'Islington Avenue, Humber Valley Village\n']
['M1B\n', 'Scarborough\n', 'Malvern, Rouge\n']
['M2B\n', 'Not assigned\n', 'Not assigned\n']
['M3B\n', 'North York\n', 'Don Mills\n']
['M4B\n', 'East York\n', 'Parkview Hill, Woodbine Gardens\n']
['M5B\n', 'Downtown Toronto\n', 'Garden District, Ryerson\n']
['M6B\n', 'North York\n', 'Glencairn\n']
['M7B\n', 'Not assigned\n', 'Not assigned\n']
['M8B\n', 'Not assigned\n', 'Not assigned\n']
['M9B\n', 'Etobicoke\n', 'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale\n']


In [2]:
# reading HTML to pandas dataframe

In [3]:
import pandas as pd
data = []
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)
for df in dfs:
    data.append(df)

table = pd.DataFrame(data[0])


In [4]:
table.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
#cleaning the data

In [6]:
# removing the cells with bouriogh not assigned

In [7]:
table.drop(table.index[table.Borough=='Not assigned'],inplace=True)

<h3>Reading into dataframe</h3>

In [8]:
table.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h3>combining rows with same postal code</h3>

In [10]:
final_table=table.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(','.join).reset_index()

<h3>for cells having a borough but a not assigned neighbourhood, replacing neighbourhood with a bourough name</h3>

In [12]:
final_table.loc[final_table['Neighbourhood'] == 'Not assigned']

,Postal Code,Borough,Neighbourhood


In [13]:
final_table

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


<h3>loading csv with lat and long info</h3>

In [15]:
lat_lon=pd.read_csv('Geospatial_Coordinates.csv')

<h3>joining of two dataframes</h3>

In [17]:
new_table=lat_lon.set_index('Postal Code').join(final_table.set_index('Postal Code'))
new_table=new_table.reset_index()

In [18]:
new_table

,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
...,...,...,...,...,...
98,M9N,43.706876,-79.518188,York,Weston
99,M9P,43.696319,-79.532242,Etobicoke,Westmount
100,M9R,43.688905,-79.554724,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,43.739416,-79.588437,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [19]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import folium

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\mriga\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.9.2                |   py38haa244fe_0         3.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  conda                                4.9.0-py38h9bdc248_1 --> 4.9.2-py38haa244fe_0




geographiclib-1.50   | 34 KB    

<h3>visualizing the neighbourhoods using geopy</h3>

In [20]:
address = 'Toronto, ontario'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [23]:
map_toronto = folium.Map(location = [latitude,longitude], zoom_start=10)

# Add markers to map
for lat, lon, neighb, bor in zip(new_table['Latitude'], new_table['Longitude'], new_table['Neighbourhood'], new_table['Borough']):
    label = '{},{}'.format(neighb,bor)
    label = folium.Popup(label, parse_html =True)
    folium.CircleMarker([lat,lon],
                        radius = 5,
                        Popup = label,
                        color = 'blue',
                        fill = True,
                        fill_color = '#3186cc',
                        fill_opacity = 0.7,
                        parse_html =False).add_to(map_toronto)
    
map_toronto

In [24]:
#lets examine north york toronto

In [66]:
 north_york= new_table[new_table['Borough'] == 'North York'].reset_index(drop=True)
north_york

,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M2H,43.803762,-79.363452,North York,Hillcrest Village
1,M2J,43.778517,-79.346556,North York,"Fairview, Henry Farm, Oriole"
2,M2K,43.786947,-79.385975,North York,Bayview Village
3,M2L,43.757490,-79.374714,North York,"York Mills, Silver Hills"
4,M2M,43.789053,-79.408493,North York,"Willowdale, Newtonbrook"
5,M2N,43.770120,-79.408493,North York,"Willowdale, Willowdale East"
6,M2P,43.752758,-79.400049,North York,York Mills West
7,M2R,43.782736,-79.442259,North York,"Willowdale, Willowdale West"
8,M3A,43.753259,-79.329656,North York,Parkwoods
9,M3B,43.745906,-79.352188,North York,Don Mills


In [60]:
address = 'North York, ontario'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York Toronto are 43.7543263, -79.44911696639593.


In [62]:
map_york = folium.Map(location = [latitude,longitude], zoom_start=10)

# Add markers to map
for lat, lon, neighb, bor in zip(north_york['Latitude'], north_york['Longitude'],north_york['Neighbourhood'], north_york['Borough']):
    label = '{},{}'.format(neighb,bor)
    label = folium.Popup(label, parse_html =True)
    folium.CircleMarker([lat,lon],
                        radius = 5,
                        Popup = label,
                        color = 'blue',
                        fill = True,
                        fill_color = '#3186cc',
                        fill_opacity = 0.7,
                        parse_html =False).add_to(map_york)
    
map_york

<h3>defining foursqaure credentials</h3>

In [64]:
CLIENT_ID = 'ER21BIC3MMJAXGZC53QHK2IKP5GWA0CGS3ZRZN4M4CDOW32G' # your Foursquare ID
CLIENT_SECRET = '24WGVOJELWKWGFZXY4AYJG31G4MAJFAO1XXMFJTIE2NQMQQR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ER21BIC3MMJAXGZC53QHK2IKP5GWA0CGS3ZRZN4M4CDOW32G
CLIENT_SECRET:24WGVOJELWKWGFZXY4AYJG31G4MAJFAO1XXMFJTIE2NQMQQR


<h3>lets explore first neighbourgood in north york</h3>

In [67]:
north_york.loc[12,'Neighbourhood']

'Northwood Park, York University'

<h3>getting neighbourhood latitude and longitude values</h3>

In [70]:
neighborhood_latitude = north_york.loc[12, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = north_york.loc[12, 'Longitude'] # neighborhood longitude value

neighborhood_name = north_york.loc[12, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Northwood Park, York University are 43.7679803, -79.48726190000001.


<h3>lets get top 75 venues that are in Malvern and rouge within a radius of 500 meters</h3>

In [72]:
LIMIT = 75
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ER21BIC3MMJAXGZC53QHK2IKP5GWA0CGS3ZRZN4M4CDOW32G&client_secret=24WGVOJELWKWGFZXY4AYJG31G4MAJFAO1XXMFJTIE2NQMQQR&v=20180605&ll=43.7679803,-79.48726190000001&radius=500&limit=75'

In [73]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ff61a89a6e8650066861247'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 8,
  'suggestedBounds': {'ne': {'lat': 43.7724803045, 'lng': -79.48104210959674},
   'sw': {'lat': 43.763480295499996, 'lng': -79.49348169040327}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d3599d2eefa8cfa141d56b3',
       'name': 'MUSE Massage Spa',
       'location': {'address': '13-1290 Finch Ave. W.',
        'crossStreet': 'at Keele St.',
        'lat': 43.7656861872247,
        'lng': -79.4893183666046,
        'labeledLatLngs': [{'label': 'display',
       

In [74]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

<h3> now we are ready to clean the json and structure it into pandas dataframe</h3>

In [76]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(5)

<ipython-input-76-12b55a922158>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,MUSE Massage Spa,Massage Studio,43.765686,-79.489318
1,Carribean Heat,Caribbean Restaurant,43.764155,-79.490227
2,Tim Hortons,Coffee Shop,43.764289,-79.488790
3,Fox & Fiddle,Bar,43.763795,-79.488497
4,Bad Boy Furniture - North York,Furniture / Home Store,43.764314,-79.486588


In [77]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


<h3> lets create a function to repeat the same process to all the neighbourhood in north york</h3>

In [79]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [82]:
york_venues = getNearbyVenues(names=north_york['Neighbourhood'],
                                   latitudes=north_york['Latitude'],
                                   longitudes=north_york['Longitude']
                                  )

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Manor, Lawrence Heights
Glencairn
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Humberlea, Emery


<h3>checking the size of resulting dataframe</h3>

In [85]:
print(york_venues.shape)
york_venues.head()

(242, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,New York Fries,43.803664,-79.363905,Fast Food Restaurant
1,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
2,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
3,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
4,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run


<h3> lets check how many venues were retured for each neighborhood</h3>

In [89]:
york_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Don Mills,25,25,25,25,25,25
Downsview,15,15,15,15,15,15
"Fairview, Henry Farm, Oriole",69,69,69,69,69,69
Glencairn,6,6,6,6,6,6
Hillcrest Village,5,5,5,5,5,5
Humber Summit,3,3,3,3,3,3


<h3> lets find how many unique categories can be curated by all the returned venues</h3>

In [91]:
print('There are {} uniques categories.'.format(len(york_venues['Venue Category'].unique())))

There are 100 uniques categories.


<h3>analyzing each neighborhood for their top venues</h3>

In [93]:
# one hot encoding
york_onehot = pd.get_dummies(york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
york_onehot['Neighborhood'] = york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [york_onehot.columns[-1]] + list(york_onehot.columns[:-1])
york_onehot = york_onehot[fixed_columns]

york_onehot.head(10)

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Fairview, Henry Farm, Oriole",0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h3>Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category</h3>

In [97]:
york_grouped = york_onehot.groupby('Neighborhood').mean().reset_index()
york_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.095238,...,0.000000,0.047619,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.045455,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.045455,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000
3,Don Mills,0.000000,0.000000,0.000000,0.04,0.000000,0.000000,0.040000,0.000000,0.000000,...,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Downsview,0.000000,0.066667,0.000000,0.00,0.000000,0.000000,0.066667,0.000000,0.066667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Fairview, Henry Farm, Oriole",0.000000,0.000000,0.014493,0.00,0.000000,0.014493,0.000000,0.028986,0.028986,...,0.000000,0.000000,0.014493,0.000000,0.000000,0.014493,0.014493,0.014493,0.000000,0.014493
6,Glencairn,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.166667,0.000000,...,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Hillcrest Village,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Humber Summit,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Humberlea, Emery",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


<h3>lets print each neighbourhood with their top 3 venues</h3>

In [98]:
num_top_venues = 3

for hood in york_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = york_grouped[york_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
         venue  freq
0         Bank  0.10
1  Coffee Shop  0.10
2  Gas Station  0.05


----Bayview Village----
                venue  freq
0  Chinese Restaurant  0.25
1                Café  0.25
2                Bank  0.25


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.09
1         Coffee Shop  0.09
2      Sandwich Place  0.09


----Don Mills----
                 venue  freq
0                  Gym  0.12
1           Beer Store  0.08
2  Japanese Restaurant  0.08


----Downsview----
           venue  freq
0  Grocery Store  0.20
1           Park  0.13
2           Bank  0.07


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.13
1           Coffee Shop  0.07
2  Fast Food Restaurant  0.06


----Glencairn----
                 venue  freq
0          Pizza Place  0.33
1  Japanese Restaurant  0.17
2               Bakery  0.17


----Hillcrest Village